# Data Mining
## Practical Assignment 1

### Bär Halberkamp (10758380) & Steven Raaijmakers (10804242)

In [25]:
import numpy as np
import pandas as pd

### Part 1: Loading the data into a Pandas Data Frame [0.5 pts]

In [26]:
features = ['1stFlrSF','2ndFlrSF','BedroomAbvGr','TotalBsmtSF',
            'LotFrontage','LotArea','MasVnrArea','BsmtFinSF1',
            'BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','GrLivArea']
target = ['SalePrice']

# read csv
df = pd.read_csv('house_prices.csv',sep=",", header = 0, 
                 usecols=target + features)

# features dataframes

X = (df[features]).values

# target dataframe
y = np.array([df['SalePrice']])
y = y.reshape(-1, 1)

### Part 2: Split the data into a training set and a test set [0.5 pts]

In [27]:
# Split the data into a training and a test set. Use a  70%-30% split.
# Your code goes here.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print("Train set size:", len(X_train))
print("Test set size:", len(X_test))
# Print the number of examples in the training set and the test set
# Your code goes here.

Train set size: 1022
Test set size: 438


### Part 3: Linear Regression [2 pts]

1. Train a linear regression model
2. Compare the importance of features based on the parameters $\theta$ of the model
3. Compute goodness-of-fit, $R^2$, and error, RMSE

** Missing Data **: Often the data you are considering is incomplete. For example there can be the case that the people that collected the training data forgot to save the number of bedrooms for some example house in the training of test set. In this case, if you look into the datasets you will find the value *NaN*. This is not a real value, hence Linear Regression cannot handle it.

The question is how can we handle missing data. There are many ways to do so, some more sophisticated than others. Here we will use a simple approach. This simple approach fills in the missing values, i.e. replaces the *NaN* by the median of the corresponding feature. E.g. if there is a *NaN* value for the number of bedrooms, this *NaN* value will be replaced by the median number of bedrooms in all other example houses in the data.

In [28]:
# Fill in the missing data in the dataset (i.e. replace NaN values) 

from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='median', axis=0, verbose=0, copy=False)
imputer.fit(X_train) # replace X_train with the name of the training features array in your code
imputer.transform(X_train) # replace X_train with the name of the training features array in your code
imputer.transform(X_test) # replace X_test with the name of the test features array in your code

array([[  896.,   448.,     3., ...,   896.,   896.,  1344.],
       [ 1575.,   626.,     4., ...,   697.,   697.,  2201.],
       [  975.,   975.,     3., ...,   326.,   975.,  1950.],
       ..., 
       [ 1493.,     0.,     3., ...,   484.,  1478.,  1493.],
       [ 1117.,     0.,     3., ...,   480.,  1029.,  1117.],
       [ 2036.,     0.,     3., ...,    80.,  2136.,  2036.]])

In [29]:
# Train a linear regression model
# Your code goes here
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)

# Print the parameters theta
# Your code goes here
print("Theta_0: %f.0" % (lr.intercept_))
for i, t in enumerate(lr.coef_[0]):
    print("Theta_" + str(i+1) + ":", t)

Theta_0: 26222.993607.0
Theta_1: 107.732463261
Theta_2: 114.194416786
Theta_3: -18139.0178311
Theta_4: 20.1267765262
Theta_5: 76.0628983581
Theta_6: 0.208818526485
Theta_7: 59.8591060606
Theta_8: 16.9102643152
Theta_9: -5.50597165218
Theta_10: 8.72248386281
Theta_11: 20.1267765263
Theta_12: -17.0656235595


**Question 1**. Can you identify the 5 features that are the most important for the predicting the house price? If yes, identify and print these top-5 features. If no, explain the problem and solve it so you can answer the question.

In [30]:
# get top 5 highest abs thetas
top = list(reversed(sorted([abs(i) for i in lr.coef_[0]])))[:5]
abs_f = [abs(i) for i in lr.coef_[0]]
top_thetas = [list(abs_f).index(i) for i in top]

print([features[i] for i in top_thetas])

['BedroomAbvGr', '2ndFlrSF', '1stFlrSF', 'LotFrontage', 'MasVnrArea']


###### Root mean squared error

Implement the root mean squared error function, without using the scikit-learn mean_squared_error.

In [31]:
def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

**Question 2**. What is the $R^2$ goodness-of-fit and the RMSE of your model when (a) computing them on the training set, and (b) computing them on the test set? Which one shall you trust?

In [32]:
# your code goes here
print("R^2")
print("Train = %f" % lr.score(X_train, y_train))
print("Test = %f" % lr.score(X_test, y_test))

print("\nRMSE")
print("Train = %f" % rmse(y_train, lr.predict(X_train)))
print("Test = %f" % rmse(y_test, lr.predict(X_test)))

R^2
Train = 0.665784
Test = 0.652254

RMSE
Train = 46491.488090
Test = 45355.460856


The model is already fitted to the training set, so to check the perfomance the test set is more trustable.

### Part 4. Adding features [3 pts]
1. Add a number of features by including polynomials and interactions of different degree
2. Train and test the different linear regression models over the data
3. Test whether increasing the complexity of the model overfits the data

Consider the original dataset that was loaded into Pandas, and then turned into a Numpy array from Part 1.

In [33]:
# Implement a function that constructs additional features by considering the polynomials of the original features
# along with their interactions. degree is the degree of the polynomial.

from sklearn.preprocessing import PolynomialFeatures

# Implement a function that constructs additional features by considering the polynomials of the original features
# along with their interactions. degree is the degree of the polynomial.

def polynomial(X, degree):
    X = X.reshape(-1, 1)
    poly = PolynomialFeatures(degree=degree, include_bias=False).fit(X)
    X_poly = poly.transform(X)
    return X_poly

In [34]:
from sklearn.preprocessing import RobustScaler

# Generate polynomial dataset (both training and test) of degrees 1, 2, 3, 4, 5
X_train_polys = [polynomial(X_train, p) for p in range(1, 6)]
X_test_polys = [polynomial(X_test, p) for p in range(1, 6)]

# Scale all features using the RobustScaler
robust_scaler = RobustScaler()
scaled_train = [robust_scaler.fit_transform(x) for x in X_train_polys]
scaled_test = [robust_scaler.fit_transform(x) for x in X_test_polys]

# Compute and print RMSE using your code above on the training set and on the test set
rmse_train = [rmse(y_train, x) for x in scaled_train]
rmse_test = [rmse(y_test, x) for x in scaled_test]

print("RSMEs for polynomials with degree [1, 2, 3, 4, 5]: {} (train) {} (test)".format(rmse_train, rmse_test))

# Compute and print R^2 on the training set and on the test set
r2_train = [x.score(X_train, y_train) for x in scaled_train]
r2_test = [x.score(X_test, y_test) for x in scaled_test]

print("R^2s for polynomials with degree [1, 2, 3, 4, 5]: {} (train) {} (test)".format(r2_train, r2_test))

# Generate a plot with the x-axis representing the complexity of the model (i.e. the degree of the polynomial features)
# Make the degree range from 1 to 5. The y-axis should represent the RMSE. Plot a line for degree = 1, 2, 3, 4, and 5
# for the training error and the test error.
plt.plot(range(1, 6), rmse_train, label="Training set")
plt.plot(range(1, 6), rmse_test, label="Test set")
plt.legend()
plt.show()

ValueError: operands could not be broadcast together with shapes (12264,1) (1022,1) 

** Question 3**. Write your conclusions regarding the performance of the models of increasing complexity

your answer goes here

### Part 5. Regularization [2pts]

1. Feature selection using regularization
2. Training/validation/test split
3. Find the optimal parameter for the regularizer
4. Compare linear regression with and without regularization

Use the 2 degree polynomial features constructed above.



In [ ]:
from sklearn.linear_model import Lasso, Ridge

# Use regularization for feature selection; choose the \lambda parameter (i.e. the alpha in python) equal to 500
model = Ridge(500).fit(X_train, y_train)

print("RMSE (training): {} (no regularization), {} (with regularization)".format(rmse(y_train, lr.predict(X_train)), rmse(y_train, model.predict(X_train))))
print("RMSE (test): {} (no regularization), {} (with regularization)".format(rmse(y_test, lr.predict(X_test)), rmse(y_test, model.predict(X_test))))


# Identify how many features were selected (i.e. had a non-zero parameter \theta) by the regularized model
# your code goes here
print("Number of features used: {}".format(np.sum(model.coef_ != 0)))

**Question 4**: Write your conclusions regarding the performance of the two models answering the questions above.

Regularization seems to not improve the model's fit for neither the training or test set.

In [ ]:
import matplotlib.pyplot as plt

# Compare the coefficients (i.e. the parameters \theta) of the linear regression models with and without regularization.
# Plot them in a graph, where the x-axis is the index of a coefficient while the y-axis the magnitute of it.
yvals = (list(lr.coef_[0][:]), list(model.coef_[0][:]))
yvals[0].insert(0, lr.intercept_)
yvals[0].insert(0, model.intercept_)
plt.plot(range(len(lr.coef_[0]) + 2) ,yvals[0], linestyle='--', marker='o', markersize=4, linewidth=2, label="no regularization")
plt.plot(range(len(model.coef_[0])) ,yvals[1], linestyle='--', marker='o', markersize=4, linewidth=2, label="with regularization")
plt.legend()
plt.show()

print(yvals)

**Question 5**. Write your conclusions regarding the coeeficients of the two models based on the plot. 

Regularization seems to prioritize a different feature, which can explain it being a worse fit. It's unclear why a different feature is chosen, and a plot for different values of alpha could confirm whether that is the cause of the problems with the regularized model

### Part 6: Categorical features [2 pts]

Consider now the entire dataset, without selecting specific features as you did in Part 1. Features in this dataset are both *numerical* and *categorical*.

**Question 6**. Look at the description of the features. Which features need to be converted into numerical? Provide 5 features as examples.

MSZoning, Alley, Utilities, HouseStyle, Foundation

**Question 7**. Which features need to be converted into integers and which into one-hot encoding? Provide 3 features for each case. Explain your decision.

Integers:
BsmtExposure
BsmtQual
BsmtCond

One-hot encoding:
Roof material
RoofStyle
Neighborhood

Transform all categorical features into an one-hot encoding representation.

In [35]:
df2 = pd.read_csv('house_prices.csv',sep=",", header = 0)

# Transform all categorical features into one-hot encoding.
df2 = pd.get_dummies(df2)
   
# Split the dataset into training, validation and test sets, using 60%, 20%, and 20% splits.
y2 = np.array([df['SalePrice']])
y2 = y.reshape(-1, 1)

X2 = np.array(df2[df2.columns.difference(['SalePrice'])])        

# split data into train + validation + test 
X_trainval2, X_test2, y_trainval2, y_test2 = train_test_split(X2, y2, random_state=1, test_size=0.20)
X_train2, X_valid2, y_train2, y_valid2 = train_test_split(X_trainval2, y_trainval2, random_state=1, test_size=0.25)

In [36]:
imputer2 = Imputer(missing_values='NaN', strategy='median', axis=0, verbose=0, copy=False)
imputer2.fit(X_train2)
imputer2.transform(X_train2)
imputer2.transform(X_test2)
imputer2.transform(X_valid2)

array([[ 1391.,     0.,     0., ...,  2004.,  2004.,  2008.],
       [ 1173.,     0.,     0., ...,  1974.,  1974.,  2010.],
       [ 1324.,     0.,     0., ...,  2006.,  2006.,  2009.],
       ..., 
       [  902.,   808.,     0., ...,  1921.,  1950.,  2010.],
       [  698.,     0.,     0., ...,  1947.,  2008.,  2009.],
       [ 1022.,  1038.,   168., ...,  1999.,  1999.,  2009.]])

In [37]:
# Normalize all features based on the training and validation set combined.
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train2)
X_train_scaled = min_max_scaler.transform(X_train2)
min_max_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_valid2)
X_valid_scaled = min_max_scaler.transform(X_valid2)

# Train a linear regression model on the training and validation set combined. 
X_train_valid = np.concatenate((X_train_scaled, X_valid_scaled), axis=0)
y_train_valid = np.concatenate((y_train2, y_valid2), axis=0)
lr2 = LinearRegression().fit(X_train_valid, y_train_valid)

In [38]:
# Train a regularized model (use Ridge regularization). To decide the parameter alpha, use a range of values to train,
# the model. Test its performance on the validation set. Output the RMSE for different values of alpha, and choose the
# best value.
# your code goes here

from sklearn.linear_model import Ridge

alphas = list(np.arange(1, 20, 1))
r2s = []

for alpha in alphas:
    ridge = Ridge(alpha).fit(X_train2, y_train2)
    r2 = ridge.score(X_valid2, y_valid2)
    rmse = np.sqrt(((y_valid2 - ridge.predict(X_valid2)) ** 2).mean())
    r2s.append(r2)
    print(alpha, "\tR^2:", r2, "\tRMSE:", rmse)


best_alpha = alphas[r2s.index(max(r2s))]
print("\nBest alpha:", best_alpha)

1 	R^2: 0.888786741421 	RMSE: 24452.996024
2 	R^2: 0.897614369455 	RMSE: 23462.4466939
3 	R^2: 0.901060522705 	RMSE: 23064.2108739
4 	R^2: 0.902986316924 	RMSE: 22838.6427213
5 	R^2: 0.904232389428 	RMSE: 22691.4955451
6 	R^2: 0.905097361045 	RMSE: 22588.7884839
7 	R^2: 0.905718926274 	RMSE: 22514.6942824
8 	R^2: 0.906171773522 	RMSE: 22460.5583389
9 	R^2: 0.906501095562 	RMSE: 22421.1072114
10 	R^2: 0.906736436001 	RMSE: 22392.8720262
11 	R^2: 0.906898235555 	RMSE: 22373.439306
12 	R^2: 0.907001256646 	RMSE: 22361.0572952
13 	R^2: 0.907056521539 	RMSE: 22354.4122318
14 	R^2: 0.907072480069 	RMSE: 22352.4930069
15 	R^2: 0.90705574982 	RMSE: 22354.5050371
16 	R^2: 0.907011605296 	RMSE: 22359.8131215
17 	R^2: 0.906944312663 	RMSE: 22367.9021877
18 	R^2: 0.906857365632 	RMSE: 22378.3495267
19 	R^2: 0.906753655886 	RMSE: 22390.8046568

Best alpha: 14


In [39]:
# Train the regularized model (use Ridge regularization) with the best alpha parameter found above on the traing and 
# validation set combined.
# your code goes here
ridge = Ridge(best_alpha).fit(X_train_valid, y_train_valid)

In [40]:
# Compare the performance of the linear regression with and without regularization
# your code goes here
lr2_score = lr2.score(X_test2, y_test2)
reg_score = ridge.score(X_test2, y_test2)

print("Lr2: %f" % lr2_score)
print("Reg: %f" % reg_score)

Lr2: -396744797980444393472.000000
Reg: -10931278.086818


**Question 6**: What are your conclusions for the two model compared?

The regularized model has a better score (R^2) because it is more close to zero, thus the prediction using the regularized model will be more accurate.